# Linear Regression

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
 Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [0]:
# Import Spark Session
from pyspark.sql import SparkSession

In [0]:
# Instance spark 
spark = SparkSession.builder.appName('lr_consulting').getOrCreate()

In [0]:
# Get data from csv
data = spark.sql('SELECT * FROM cruise_ship_info')

In [0]:
# Print Schema (format of columns)
data.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [0]:
# Print dataset 
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 20 rows

In [0]:
# Count unique Cruise Line
data.select('Cruise_line').distinct().count()

Out[17]: 20

In [0]:
# Indexer Cruise Line
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_indexed")
indexed_data = indexer.fit(data).transform(data)

In [0]:
# Show indexed dataset
indexed_data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_indexed|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
only showing top 20 rows

In [0]:
# Show columns start in index 2 
indexed_data.columns[2:]

Out[15]: ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_indexed']

In [0]:
# Select and create final dataset
final_df = indexed_data.select(indexed_data.columns[2:])

In [0]:
final_df.show()

+---+------------------+----------+------+------+-----------------+----+-------------------+
Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_indexed|
+---+------------------+----------+------+------+-----------------+----+-------------------+
 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|
 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|
 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|
 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|
 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|
 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|
 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|
 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|
 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|
 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
+---+------------------+----------+------+------+-----------------+----+-------------------+
only showing top 20 rows

In [0]:
final_df.columns

Out[22]: ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_indexed']

In [0]:
# Import Verctors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
# Assembler 
assembler = VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density','Cruise_line_indexed'], outputCol='features')

In [0]:
# get final vector data
final_vec_data = assembler.transform(final_df) 

In [0]:
# Print final dataset
final_vec_data.show(truncate=False)

+---+------------------+----------+------+------+-----------------+----+-------------------+--------------------------------------------------+
Age|Tonnage |passengers|length|cabins|passenger_density|crew|Cruise_line_indexed|features |
+---+------------------+----------+------+------+-----------------+----+-------------------+--------------------------------------------------+
6 |30.276999999999997|6.94 |5.94 |3.55 |42.64 |3.55|16.0 |[6.0,30.276999999999997,6.94,5.94,3.55,42.64,16.0]|
6 |30.276999999999997|6.94 |5.94 |3.55 |42.64 |3.55|16.0 |[6.0,30.276999999999997,6.94,5.94,3.55,42.64,16.0]|
26 |47.262 |14.86 |7.22 |7.43 |31.8 |6.7 |1.0 |[26.0,47.262,14.86,7.22,7.43,31.8,1.0] |
11 |110.0 |29.74 |9.53 |14.88 |36.99 |19.1|1.0 |[11.0,110.0,29.74,9.53,14.88,36.99,1.0] |
17 |101.353 |26.42 |8.92 |13.21 |38.36 |10.0|1.0 |[17.0,101.353,26.42,8.92,13.21,38.36,1.0] |
22 |70.367 |20.52 |8.55 |10.2 |34.29 |9.2 |1.0 |[22.0,70.367,20.52,8.55,10.2,34.29,1.0] |
15 |70.367 |20.52 |8.55 |10.2 |34.29 |9.2 |1.0 |[15.0,70.367,20.52,8.55,10.2,34.29,1.0] |
23 |70.367 |20.56 |8.55 |10.22 |34.23 |9.2 |1.0 |[23.0,70.367,20.56,8.55,10.22,34.23,1.0] |
19 |70.367 |20.52 |8.55 |10.2 |34.29 |9.2 |1.0 |[19.0,70.367,20.52,8.55,10.2,34.29,1.0] |
6 |110.23899999999999|37.0 |9.51 |14.87 |29.79 |11.5|1.0 |[6.0,110.23899999999999,37.0,9.51,14.87,29.79,1.0]|
10 |110.0 |29.74 |9.51 |14.87 |36.99 |11.6|1.0 |[10.0,110.0,29.74,9.51,14.87,36.99,1.0] |
28 |46.052 |14.52 |7.27 |7.26 |31.72 |6.6 |1.0 |[28.0,46.052,14.52,7.27,7.26,31.72,1.0] |
18 |70.367 |20.52 |8.55 |10.2 |34.29 |9.2 |1.0 |[18.0,70.367,20.52,8.55,10.2,34.29,1.0] |
17 |70.367 |20.52 |8.55 |10.2 |34.29 |9.2 |1.0 |[17.0,70.367,20.52,8.55,10.2,34.29,1.0] |
11 |86.0 |21.24 |9.63 |10.62 |40.49 |9.3 |1.0 |[11.0,86.0,21.24,9.63,10.62,40.49,1.0] |
8 |110.0 |29.74 |9.51 |14.87 |36.99 |11.6|1.0 |[8.0,110.0,29.74,9.51,14.87,36.99,1.0] |
9 |88.5 |21.24 |9.63 |10.62 |41.67 |10.3|1.0 |[9.0,88.5,21.24,9.63,10.62,41.67,1.0] |
15 |70.367 |20.52 |8.55 |10.2 |34.29 |9.2 |1.0 |[15.0,70.367,20.52,8.55,10.2,34.29,1.0] |
12 |88.5 |21.24 |9.63 |11.62 |41.67 |9.3 |1.0 |[12.0,88.5,21.24,9.63,11.62,41.67,1.0] |
20 |70.367 |20.52 |8.55 |10.2 |34.29 |9.2 |1.0 |[20.0,70.367,20.52,8.55,10.2,34.29,1.0] |
+---+------------------+----------+------+------+-----------------+----+-------------------+--------------------------------------------------+
only showing top 20 rows

In [0]:
# Select X like features and Y like crew
filtered_data = final_vec_data.select('features', 'crew')

In [0]:
filtered_data.show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [0]:
# Split data into train and test(70% to train and 30% to test)
train_data, test_data = filtered_data.randomSplit([0.7,0.3])

In [0]:
# Describe train
train_data.describe().show()

+-------+------------------+
summary| crew|
+-------+------------------+
 count| 117|
 mean|7.8248717948718065|
 stddev|3.1952533803215415|
 min| 0.59|
 max| 13.6|
+-------+------------------+

In [0]:
# Describe test
test_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 41|
 mean|7.706585365853658|
 stddev|4.307958106664977|
 min| 0.59|
 max| 21.0|
+-------+-----------------+

In [0]:
# Import linear regression
from pyspark.ml.regression import LinearRegression

In [0]:
# Build model
lr = LinearRegression(featuresCol='features', labelCol='crew',predictionCol='prediction')

In [0]:
# Train model
lrModel = lr.fit(train_data)

In [0]:
# Evaluate model with test data
model_results = lrModel.evaluate(test_data)

In [0]:
# MSE
model_results.meanSquaredError

Out[39]: 1.6856075735875804

In [0]:
# RMSE
model_results.rootMeanSquaredError

Out[40]: 1.2983095060838077

In [0]:
# Square R
model_results.r2

Out[41]: 0.9069026333435443

In [0]:
# "Manual" prediction with unkown data 
predict_data = lrModel.transform(test_data.select('features'))

In [0]:
test_data.select('crew').show()

+-----+
 crew|
+-----+
 21.0|
 9.0|
11.09|
 11.5|
 9.87|
 12.0|
 11.0|
 6.36|
 8.58|
12.53|
 7.6|
 8.0|
 8.48|
 19.1|
11.85|
 4.0|
 8.5|
 11.5|
 3.73|
 10.0|
+-----+
only showing top 20 rows

In [0]:
predict_data.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[4.0,220.0,54.0,1...|20.019550546502227|
[6.0,90.0,20.0,9....|10.156681906793953|
[6.0,93.0,23.94,9...|10.563601457174986|
[6.0,110.23899999...|10.982379436421564|
[7.0,89.6,25.5,9....|11.110180108718957|
[7.0,116.0,31.0,9...|12.376920886900576|
[8.0,91.0,22.44,9...|10.119373571465973|
[10.0,68.0,10.8,7...| 6.667070090850345|
[10.0,90.09,25.01...| 8.937078371210427|
[10.0,151.4,26.2,...| 11.20362690118468|
[11.0,58.6,15.66,...| 7.533832454319816|
[11.0,85.0,18.48,...| 8.844324910243918|
[11.0,90.09,25.01...| 8.921080579923789|
[11.0,110.0,29.74...|11.866068342470884|
[12.0,138.0,31.14...|12.601705112611594|
[13.0,30.27699999...| 4.082989546455975|
[13.0,76.0,18.74,...| 8.797693676784421|
[13.0,101.509,27....|10.884418820333181|
[14.0,30.27699999...| 3.458936346309115|
[14.0,101.509,27....|10.403262437608351|
+--------------------+------------------+
only showing top 20 rows